# DFPT Phonon Step (PH.X)

In [1]:
### Import libraries
from ipywidgets import Style, Box, Layout, Label, IntText, FloatText, Dropdown
%run styles.ipynb
%run color_changer.ipynb

## PH.X Inputs

In [2]:
# ph.x inputs

#qpoints in 100 crystal direction, for sampling periodic cell in reciprocal space in the phonon calculation
nq1 = IntText(name="nq1", value= 3,layout=input_layout(30) )

#qpoints in 010 crystal direction, for sampling periodic cell in reciprocal space in the phonon calculation
nq2 = IntText(name="nq2", value= 3, layout=input_layout(30))
# qpoints in 001 crystal direction, for sampling periodic cell in reciprocal space in the phonon calculation
nq3 = IntText(name="nq3", value= 3, layout=input_layout(30))

# convergence threshold for phonon calculation
self_consistency_threshold = FloatText(name="Self-Consistency Threshold", value=1.0e-12, layout=input_layout(30))

# option for recovering from previous failed/timed out calculation
recover = Dropdown(name="recover", value="false", options=["true","false"], layout=input_layout(30))

form_items = [
    Box([Label(value='nq1'), nq1], layout=form_item_layout()),
    Box([Label(value='nq2'), nq2], layout=form_item_layout()),
    Box([Label(value='nq3'), nq3], layout=form_item_layout()),
    Box([Label(value='Self-Consistency Threshold'), self_consistency_threshold], layout=form_item_layout()),
    Box([Label(value='Recover'), recover], layout=form_item_layout())
]

ph_x_box = Box(form_items, layout=box_layout(30))

ph_x_box


Box(children=(Box(children=(Label(value='nq1'), IntText(value=3, layout=Layout(width='30%'))), layout=Layout(d…

## NON USER INPUTS

In [3]:
#input parameters
#&inputph                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
search_sym = '.false.'   
ph_cores = 20
ph_walltime = '00:05:00'

## Bind inputs to outputs

In [4]:
def bind_PH_X_inputs(self):
    
    ph_inputs = {
        'material_prefix': material_prefix.value,
        'nq1': nq1.value,
        'nq2': nq2.value,
        'nq3': nq3.value,
        'tr2_ph': self_consistency_threshold.value,
        'recover': recover.value,
        'search_sym': search_sym
    }

    build_PH_Input(ph_inputs, material_prefix.value)

In [5]:
def build_PH_Input(ph_inputs, material_prefix):
    print('BUILDING PH INPUT FILE')
    ph_name = 'ph-%s.in' % material_prefix #assigns ph input file to variable ph_name 

    ph_name = 'ph-%s.in' % material_prefix #assigns ph input file to variable ph_name 

    input_file = '''
    --
    &inputph
    prefix = '{material_prefix}'
    fildvscf = 'dvscf'
    ldisp = .true.
    fildyn = '{material_prefix}.dyn'
    nq1 = {nq1}
    nq2 = {nq2}
    nq3 = {nq3}
    tr2_ph = {tr2_ph}
    recover = {recover}
    search_sym = {search_sym}
    /
    '''.format(**ph_inputs) #assigns information in ''' ''' to variable inputfile

    with open(ph_name, "w") as f: #opens file pw_name
        f.write(input_file) #writes inputfile to file ph